# 记录智能体行为

在训练期间或评估智能体时，记录一集中的智能体行为并记录累积的总奖励可能会很有趣。这可以通过两个包装器实现：`RecordEpisodeStatistics` 和 `RecordVideo`，第一个跟踪剧集（Episode）数据，如总奖励、剧集长度和所用时间，第二个使用环境渲染生成智能体的 mp4 视频。

我们展示了如何为两种类型的问题应用这些包装器；第一种是记录每个剧集的数据（通常是评估），第二种是定期记录数据（用于正常训练）。

## 记录每个剧集

给定训练有素的智能体，您可能希望在评估期间记录几个剧集以了解智能体的行为。下面，我们提供了示例脚本，演示如何使用 `RecordEpisodeStatistics` 和 `RecordVideo` 进行此操作。

In [1]:
import gymnasium as gym
from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo

num_eval_episodes = 4

env = gym.make("CartPole-v1", render_mode="rgb_array")  # replace with your environment
env = RecordVideo(env, video_folder="cartpole-agent", name_prefix="eval",
                  episode_trigger=lambda x: True)
env = RecordEpisodeStatistics(env, buffer_length=num_eval_episodes)

for episode_num in range(num_eval_episodes):
    obs, info = env.reset()

    episode_over = False
    while not episode_over:
        action = env.action_space.sample()  # replace with actual agent
        obs, reward, terminated, truncated, info = env.step(action)

        episode_over = terminated or truncated
env.close()

print(f'Episode time taken: {env.time_queue}')
print(f'Episode total rewards: {env.return_queue}')
print(f'Episode lengths: {env.length_queue}')

Episode time taken: deque([0.051203, 0.041812, 0.047081, 0.023832], maxlen=4)
Episode total rewards: deque([14.0, 13.0, 18.0, 9.0], maxlen=4)
Episode lengths: deque([14, 13, 18, 9], maxlen=4)


在上述脚本中，对于 `RecordVideo` 包装器，我们指定了三个不同的变量：`video_folder` 用于指定应保存视频的文件夹（根据您的问题进行更改），`name_prefix` 用于视频本身的前缀，最后是 `episode_trigger` 以便记录每个剧集。这意味着对于环境的每个剧集，都将录制一个视频并保存为“cartpole-agent/eval-episode-x.mp4”样式。

对于 `RecordEpisodicStatistics`，我们只需要指定缓冲区长度，这是内部 `time_queue`、`return_queue` 和 `length_queue` 的最大长度。与其为每个剧集单独收集数据，我们可以使用数据队列在评估结束时打印信息。

为了加快评估环境的速度，可以使用向量环境同时并行评估 `N` 个剧集，而不是按顺序评估。

## 在训练期间记录智能体行为

在训练期间，一个智能体会进行数百或数千个剧集，因此您不能为每个剧集录制视频，但开发人员可能仍然希望了解智能体在训练的不同阶段的行为，定期在训练期间记录剧集。而对于剧集统计，知道每一集的数据会更有帮助。以下脚本提供了一个例子，展示如何在记录每一集的统计数据的同时，定期记录智能体的剧集（我们使用 Python 的 `logger`，但 {mod}`tensorboard`、{mod}`wandb` 等模块也可用）。

In [2]:
import logging

import gymnasium as gym
from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo

training_period = 250  # record the agent's episode every 250
num_training_episodes = 10_000  # total number of training episodes

env = gym.make("CartPole-v1", render_mode="rgb_array")  # replace with your environment
env = RecordVideo(env, video_folder="cartpole-agent", name_prefix="training",
                  episode_trigger=lambda x: x % training_period == 0)
env = RecordEpisodeStatistics(env)

for episode_num in range(num_training_episodes):
    obs, info = env.reset()

    episode_over = False
    while not episode_over:
        action = env.action_space.sample()  # replace with actual agent
        obs, reward, terminated, truncated, info = env.step(action)

        episode_over = terminated or truncated

    logging.info(f"episode-{episode_num}", info["episode"])
env.close()

/media/pc/data/lxw/envs/anaconda3x/envs/xxx/lib/python3.12/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /media/pc/data/lxw/ai/d2py/doc/libs/gym/cartpole-agent folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
